In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
import random
from tqdm import tqdm
import wandb
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import pandas as pd
from collections import Counter
import random

import pandas as pd

In [2]:
import pandas as pd

# Path to the CSV file
test_data_dir = '/kaggle/input/aksharantar-data/aksharantar_sampled/hin/hin_test.csv'
train_data_dir = '/kaggle/input/aksharantar-data/aksharantar_sampled/hin/hin_train.csv'
val_data_dir = '/kaggle/input/aksharantar-data/aksharantar_sampled/hin/hin_valid.csv'

train_data = pd.read_csv(train_data_dir)
val_data = pd.read_csv(val_data_dir)
test_data = pd.read_csv(test_data_dir)


# print(train_data.shape[0])
# print(val_data.shape[0])
# print(test_data.shape[0])
print(train_data.shape[0] + val_data.shape[0] + test_data.shape[0])

59389


In [3]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

def load_data(train_path, test_path, val_path):
    train_data = pd.read_csv(train_path)
    val_data = pd.read_csv(val_path)
    test_data = pd.read_csv(test_path)

#     input_texts = train_data.iloc[:, 0].tolist() + test_data.iloc[:, 0].tolist() + val_data.iloc[:, 0].tolist()
#     target_texts = train_data.iloc[:, 1].tolist() + test_data.iloc[:, 1].tolist() + val_data.iloc[:, 1].tolist()

    input_texts = train_data.iloc[:, 0].tolist()
    target_texts = train_data.iloc[:, 1].tolist()
    
    val_input_texts = val_data.iloc[:, 0].tolist()
    val_target_texts = val_data.iloc[:, 1].tolist()
    
    test_input_texts = test_data.iloc[:, 0].tolist()
    test_target_texts = test_data.iloc[:, 1].tolist()

    input_characters = sorted(list(set(''.join(input_texts))))
    target_characters = sorted(list(set(''.join(target_texts))))

    input_token_index = dict([(char, i+3) for i, char in enumerate(input_characters)])
    target_token_index = dict([(char, i+3) for i, char in enumerate(target_characters)])

    num_encoder_tokens = len(input_characters)
    num_decoder_tokens = len(target_characters)

    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])

    encoder_input_data = [[input_token_index[char] for char in txt] for txt in input_texts]
    decoder_input_data = [[target_token_index[char] for char in txt] for txt in target_texts]
    decoder_target_data = [[target_token_index[char] for char in txt[1:]] for txt in target_texts]

    encoder_input_data = pad_sequence([torch.tensor(seq) for seq in encoder_input_data], batch_first=True)
    decoder_input_data = pad_sequence([torch.tensor(seq) for seq in decoder_input_data], batch_first=True)
    decoder_target_data = pad_sequence([torch.tensor(seq) for seq in decoder_target_data], batch_first=True)

    return input_texts, target_texts, input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, encoder_input_data, decoder_input_data, decoder_target_data, val_input_texts, val_target_texts, test_input_texts, test_target_texts
#     return input_texts, target_texts, input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, encoder_input_data, decoder_input_data, decoder_target_data

# Path to the CSV files
input_texts, target_texts, input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, encoder_input_data, decoder_input_data, decoder_target_data, val_input_texts, val_target_texts, test_input_texts, test_target_texts = load_data(train_data_dir, test_data_dir, val_data_dir)

# input_texts, target_texts, input_characters, target_characters, num_encoder_tokens, num_decoder_tokens, max_encoder_seq_length, max_decoder_seq_length, input_token_index, target_token_index, encoder_input_data, decoder_input_data, decoder_target_data = load_data(train_data_dir, test_data_dir, val_data_dir)

print(input_token_index)
print(decoder_input_data[0])
print(decoder_input_data.shape)

{'a': 3, 'b': 4, 'c': 5, 'd': 6, 'e': 7, 'f': 8, 'g': 9, 'h': 10, 'i': 11, 'j': 12, 'k': 13, 'l': 14, 'm': 15, 'n': 16, 'o': 17, 'p': 18, 'q': 19, 'r': 20, 's': 21, 't': 22, 'u': 23, 'v': 24, 'w': 25, 'x': 26, 'y': 27, 'z': 28}
tensor([40, 55, 37, 66, 35, 66, 43, 54,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0])
torch.Size([51199, 20])


In [4]:
# print(encoder_input_data)
# print(encoder_input_data.shape)

In [5]:
# import torch
# import torch.nn as nn

# # Define the embedding layer
# embedding = nn.Embedding(100, 10)  # 100 is the size of the vocabulary, 10 is the size of each embedding vector

# # Define your input list
# input_list = [40, 55, 37, 20, 0, 0, 0, 0, 0, 0, 0, 0]

# # Convert the input list to a PyTorch tensor and add a batch and sequence dimension
# input_tensor = torch.LongTensor(input_list).unsqueeze(0)  # Add a batch dimension

# # Pass the input tensor through the embedding layer
# embedded = embedding(input_tensor)

# # Define RNN
# rnn = nn.RNN(input_size=10, hidden_size=5, batch_first=True)  # input_size is the size of each embedding vector

# # Pass the embedded tensor through the RNN
# out, hid = rnn(embedded)

# # Print the shapes of the output tensor and hidden state
# print("Output shape:", out.shape)
# print("Hidden state shape:", hid.shape)

# # Print the embedded tensor
# print('emb_mat:',embedded.shape)
# # print(embedded)
# print('out:',out)
# print('hid:',hid)

In [6]:
# import torch

# # Define your input list
# input_list = [40, 55, 37, 20, 0, 0, 0, 0, 0, 0, 0, 0]

# # Convert the input list to a PyTorch tensor
# input_tensor = torch.LongTensor(input_list)
# print(input_tensor)
# print(input_tensor.unsqueeze(1))
# print(input_tensor.unsqueeze(0))

# print(input_tensor.unsqueeze(1).shape)
# print(input_tensor.unsqueeze(0).shape)

# # Unsqueezing the tensor
# unsqueezed_tensor = input_tensor.unsqueeze(0)


In [7]:
# Move model and data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [8]:
# Define the Encoder class
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, num_layers, dropout, cell_type, bidirectional=False):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.dropout = nn.Dropout(dropout)

        # Embedding layer
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        # Encoder recurrent layer
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, batch_first=True)

    def forward(self, x):
        # print("encoder input shape : ", x.shape)
        embedded = self.dropout(self.embedding(x))
        # print("encoder embedding shape: ", embedded.shape)
        output, hidden = self.rnn(embedded)
        if isinstance(self.rnn, nn.LSTM):
            hn, cn = hidden
        else:
            hn, cn = hidden, None
        # print("shape of hn of encoder : ", hn.shape)
        if self.bidirectional:
            if isinstance(self.rnn, nn.LSTM):
                hn, cn = hidden
                # print("output of encoder block : ", torch.sum(hn[:-2], dim=0,keepdim=True).shape, torch.sum(cn[-2:], dim=0, keepdim=True).shape)
                return torch.sum(hn[:-2], dim=0, keepdim=True), torch.sum(cn[-2:], dim=0, keepdim=True)
            else:
                return torch.sum(hidden[:-2], dim=0, keepdim=True), None
                
        else:
            if isinstance(self.rnn, nn.LSTM):
                hn, cn = hidden
            # print("output of encoder block : ", hn[-1].unsqueeze(0).shape, cn[-1].unsqueeze(0).shape)
                return hn[-1].unsqueeze(0), cn[-1].unsqueeze(0)
            else:
                return hn[-1].unsqueeze(0), None
        

# Define the Decoder class
class Decoder(nn.Module):
    def __init__(self, output_size, embedding_size, hidden_size, num_layers, dropout, cell_type):
        super(Decoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dropout = nn.Dropout(dropout)
        self.output_size = output_size
        # print(f"encoder hidden size", hidden_size)
        # Embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)

        # Decoder recurrent layer
        if cell_type == 'RNN':
            self.rnn = nn.RNN(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        elif cell_type == 'LSTM':
            self.rnn = nn.LSTM(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        elif cell_type == 'GRU':
            self.rnn = nn.GRU(embedding_size, hidden_size, num_layers, dropout=dropout, batch_first=True)
        
        # Output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, encoder_hn, encoder_cn=None):
        # hidden = hidden.transpose(0,1)
        x = x.unsqueeze(1)
        embedded = self.dropout(self.embedding(x))

        # print(f"decoder input, x:{x.shape}, encoder_hn:{encoder_hn.shape} ,encoder_cn:{encoder_cn.shape}")
        
        if isinstance(self.rnn, nn.LSTM):
            output, hidden = self.rnn(embedded, (encoder_hn, encoder_cn))
            hn, cn = hidden
        else:
            output, hidden = self.rnn(embedded, encoder_hn)
            hn, cn = hidden, None
        # print("xxxxxxxxxxxx", output.shape, hn.shape, cn.shape)
        output = self.fc(output)
        output = output.squeeze(1)
        return output, hn, cn


# Define the Seq2Seq class
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, source, target, teacher_forcing_ratio=0.5):
        # print("fiuwhuifhwug", source.shape, target.shape)
        batch_size = source.size(0)
        target_len = target.size(1)
        target_vocab_size = self.decoder.output_size
        
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(device)
        decoder_input = target[:, 0]
        encoder_hn, encoder_cn = self.encoder(source)
        encoder_hn = torch.cat([encoder_hn for _ in range(self.decoder.num_layers)], dim=0)
        if encoder_cn is not None:
            encoder_cn = torch.cat([encoder_cn for _ in range(self.decoder.num_layers)], dim=0)
        # First input to the decoder is the <SOS> tokens

        for t in range(1, target_len):
            decoder_output, encoder_hn, encoder_cn = self.decoder(decoder_input, encoder_hn, encoder_cn)
            outputs[:, t, :] = decoder_output
            best_guess = decoder_output.argmax(1)
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = decoder_output.max(1)[1]
            decoder_input = (target[:, t] if teacher_force else top1)

        return outputs

In [9]:
# Define the dataset class
class TransliterationDataset(Dataset):
    def __init__(self, input_texts, target_texts, input_token_index, target_token_index):
        self.input_texts = input_texts
        self.target_texts = target_texts
        self.input_token_index = input_token_index
        self.target_token_index = target_token_index

        # Add missing characters to target_token_index
        for text in target_texts:
            for char in text:
                if char not in target_token_index:
                    target_token_index[char] = len(target_token_index)

    def __len__(self):
        return len(self.input_texts)

    def __getitem__(self, idx):
        input_text = self.input_texts[idx]
        target_text = self.target_texts[idx]

        # Convert text sequences to token sequences
        input_seq = [self.input_token_index[char] for char in input_text]
        target_seq = [self.target_token_index[char] for char in target_text]

        return torch.tensor(input_seq), torch.tensor(target_seq)
    

def collate_fn(batch):
    input_seqs, target_seqs = zip(*batch)
    
    # Pad input sequences
    input_seqs = pad_sequence(input_seqs, batch_first=True)

    # Pad target sequences with zeros at the beginning
    max_target_length = max(len(seq) for seq in target_seqs)
    padded_target_seqs = []
    for seq in target_seqs:
        pad_length = max_target_length - len(seq)
        padded_seq = [0] * 2 + seq.tolist()
        padded_target_seqs.append(torch.tensor(padded_seq))

    # Pad all target sequences to the same maximum length
    padded_target_seqs = pad_sequence(padded_target_seqs, batch_first=True)
    
    return input_seqs, padded_target_seqs

In [10]:
# Define model parameters
cell_type = 'LSTM'
n_enc_layers = 2
n_dec_layers = 3
hidden_layer_size = 256
num_encoder_tokens = 100
num_decoder_tokens = 100
dropout = 0.2
emb_size = 256
batch_size = 64
bidirectional=True

# Modify build_model function to accept device argument
def build_model(cell_type, n_enc_layers, n_dec_layers, hidden_layer_size, num_encoder_tokens, num_decoder_tokens, dropout, emb_size, device, bidirectional):
    encoder = Encoder(input_size=num_encoder_tokens, 
                      embedding_size=emb_size, 
                      hidden_size=hidden_layer_size, 
                      num_layers=n_enc_layers, 
                      dropout=dropout, 
                      cell_type=cell_type, 
                      bidirectional=bidirectional).to(device)

    decoder = Decoder(output_size=num_decoder_tokens,
                      embedding_size=emb_size, 
                      hidden_size=hidden_layer_size, 
                      num_layers=n_dec_layers, 
                      dropout=dropout, 
                      cell_type=cell_type).to(device)

    model = Seq2Seq(encoder, decoder).to(device)
    print(model)
    
    return model

# Load data on GPU
train_dataset = TransliterationDataset(input_texts, target_texts, input_token_index, target_token_index)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# print(train_dataset[0])
# for i ,j in train_loader:
#     print(i,j)
#     break

# Initialize model and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = build_model(cell_type, n_enc_layers, n_dec_layers, hidden_layer_size, num_encoder_tokens, num_decoder_tokens, dropout, emb_size, device, bidirectional=True)
optimizer = torch.optim.Adam(model.parameters())

# Training loop
def train(model, optimizer, train_loader, device, criterion, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        total_loss = 0
        correct_predictions = 0
        total_predictions = 0
        with tqdm(train_loader, unit="batch") as tepoch:
            tepoch.set_description(f"Epoch {epoch+1}/{num_epochs}")
            for batch_idx, (input_seqs, target_seqs) in enumerate(tepoch):
                input_seqs, target_seqs = input_seqs.to(device), target_seqs.to(device)
                optimizer.zero_grad()
                output = model(input_seqs, target_seqs)
                loss = criterion(output.view(-1, output.size(-1)), target_seqs.view(-1))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                _, predicted = torch.max(output, 2)
                correct_predictions += (predicted == target_seqs).all(dim=1).sum().item()
                total_predictions += target_seqs.size(0)  #numel()
                train_acc = correct_predictions / total_predictions
                tepoch.set_postfix(loss=total_loss/(batch_idx+1), accuracy=train_acc)
                train_loss = total_loss/len(train_loader)
            print(f"Epoch [{epoch+1}/{num_epochs}], Accuracy: {correct_predictions/total_predictions:.4f}, Loss: {total_loss/len(train_loader):.4f}")
#             wandb.log({
#                 "train_loss": train_loss,
#                 "train_accuracy": train_acc,
#                 "epoches":epoch
#             })
    return model, train_acc, train_loss

            
# Define the loss function
criterion = nn.CrossEntropyLoss()
# print(model)
# Train the model
model1, train_acc, train_loss = train(model, optimizer, train_loader, device, criterion)

Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(100, 256)
    (rnn): LSTM(256, 256, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(100, 256)
    (rnn): LSTM(256, 256, num_layers=3, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=256, out_features=100, bias=True)
  )
)


Epoch 1/5: 100%|██████████| 800/800 [00:31<00:00, 25.15batch/s, accuracy=0.0251, loss=1.34]  


Epoch [1/5], Accuracy: 0.0251, Loss: 1.3354


Epoch 2/5: 100%|██████████| 800/800 [00:30<00:00, 26.21batch/s, accuracy=0.135, loss=0.821] 


Epoch [2/5], Accuracy: 0.1347, Loss: 0.8214


...

Epoch [3/5], Accuracy: 0.2029, Loss: 0.7156


Epoch 4/5: 100%|██████████| 800/800 [00:30<00:00, 26.16batch/s, accuracy=0.248, loss=0.663]


Epoch [4/5], Accuracy: 0.2477, Loss: 0.6630


Epoch 5/5: 100%|██████████| 800/800 [00:30<00:00, 25.86batch/s, accuracy=0.284, loss=0.63] 

Epoch [5/5], Accuracy: 0.2841, Loss: 0.6297


In [11]:
def validate(model1, val_loader, device, criterion):
    model.eval()
    val_total_loss = 0
    val_correct_predictions = 0
    val_total_predictions = 0
    with torch.no_grad():
        for val_input_seqs, val_target_seqs in tqdm(val_loader, desc="Validation"):
            val_input_seqs, val_target_seqs = val_input_seqs.to(device), val_target_seqs.to(device)
            val_output = model(val_input_seqs, val_target_seqs)
            val_loss = criterion(val_output.view(-1, val_output.size(-1)), val_target_seqs.view(-1))
            val_total_loss += val_loss.item()
            _, val_predicted = torch.max(val_output, 2)
#             print(f"val_predicted: {val_predicted}  val_target_seqs: {val_target_seqs}")
            val_correct_predictions += (val_predicted == val_target_seqs).all(dim=1).sum().item()
#             print(f"val_correct_predictions: {val_correct_predictions}")
            val_total_predictions += val_target_seqs.size(0)  #numel()
    val_acc = val_correct_predictions / val_total_predictions
    val_loss = val_total_loss / len(val_loader)

    return val_loss, val_acc

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define your validation dataset
val_dataset = TransliterationDataset(val_input_texts, val_target_texts, input_token_index, target_token_index)

# Define your validation data loader
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

num_epochs = 1

# Validation loop
for epoch in range(num_epochs):
    val_loss, val_acc = validate(model, val_loader, device, criterion)
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}")


Validation: 100%|██████████| 64/64 [00:00<00:00, 70.74it/s]

Validation Loss: 0.6350, Validation Accuracy: 0.3346


In [12]:
for i,j in val_loader:
    print(i.shape,j[0])
    break

torch.Size([64, 16]) tensor([ 0,  0, 40, 25, 54,  9,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])


In [13]:
import pandas as pd
import torch
from torch.utils.data import DataLoader

def decode_indices(indices, latin_idx2token):
    decoded_string = ''
    for idx in indices:
        if idx not in (0, 1, 2):
            decoded_string += latin_idx2token[idx]
    return decoded_string

def predict(model, iterator, device):
    model.eval()
    predictions = []
    with torch.no_grad():
        for src, trg in iterator:
            src = src.to(device)
            trg = trg.to(device)
            output = model(src, trg, teacher_forcing_ratio=0.0)  # Turn off teacher forcing
            output = output.argmax(2)
            src = src.cpu().numpy()
            trg = trg.cpu().numpy()
            output = output.cpu().numpy()
            predictions.append((src,trg, output))
    return predictions

# Assuming latin_idx2token and devanagari_idx2token are dictionaries mapping indices to tokens
latin_idx2token = {idx: char for char, idx in input_token_index.items()}
devanagari_idx2token = {idx: char for char, idx in target_token_index.items()}

# Load test data
test_dataset = TransliterationDataset(test_input_texts, test_target_texts, input_token_index, target_token_index)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Make predictions
test_predictions = predict(model, test_loader, device)

# Make predictions using the prediction function
test_predictions = predict(model, test_loader, device)

# Decode and print predictions for only the first 100 words
word_count = 0
for i, (src_indices, trg_indices, output_indices) in enumerate(test_predictions):
    for j in range(src_indices.shape[0]):
        input_text = decode_indices(src_indices[j], latin_idx2token)
        actual_text = decode_indices(trg_indices[j], devanagari_idx2token)
        predicted_text = decode_indices(output_indices[j], devanagari_idx2token)
        print(f'Input Text: {input_text}, Actual Text: {actual_text}, Predicted Text: {predicted_text}')
        word_count += 1
        if word_count >= 100:
            break
    if word_count >= 100:
        break

Input Text: sikhaaega, Actual Text: सिखाएगा, Predicted Text: सिखाएगा
Input Text: learn, Actual Text: लर्न, Predicted Text: लीरर्न
Input Text: twitters, Actual Text: ट्विटर्स, Predicted Text: ट्विटर्स
Input Text: tirunelveli, Actual Text: तिरुनेलवेली, Predicted Text: तिरुनेलेवीली
Input Text: independence, Actual Text: इंडिपेंडेंस, Predicted Text: इंडीपेंडेंस
Input Text: speshiyon, Actual Text: स्पेशियों, Predicted Text: स्पेशियों
Input Text: shurooh, Actual Text: शुरूः, Predicted Text: शुरूह
Input Text: kolhapur, Actual Text: कोल्हापुर, Predicted Text: कोलहापुर
Input Text: ajhar, Actual Text: अजहर, Predicted Text: अझर
Input Text: karaar, Actual Text: क़रार, Predicted Text: करार
Input Text: anka, Actual Text: अंक, Predicted Text: एंका
Input Text: wpd, Actual Text: डब्ल्यूपीडी, Predicted Text: एवपीडी
Input Text: haashie, Actual Text: हाशिए, Predicted Text: हाशी
Input Text: glendale, Actual Text: ग्लेंडल, Predicted Text: ग्लेंडले
Input Text: udhed, Actual Text: उधेड़, Predicted Text: उधेड


In [14]:
import pandas as pd
import torch
from torch.utils.data import DataLoader

# Assuming latin_idx2token and devanagari_idx2token are dictionaries mapping indices to tokens
latin_idx2token = {idx: char for char, idx in input_token_index.items()}
devanagari_idx2token = {idx: char for char, idx in target_token_index.items()}

# Load test data
test_dataset = TransliterationDataset(test_input_texts, test_target_texts, input_token_index, target_token_index)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Make predictions
test_predictions = predict(model, test_loader, device)

# Collect predictions in a list of dictionaries
results = []

for src_indices, trg_indices, output_indices in test_predictions:
    for j in range(src_indices.shape[0]):
        input_text = decode_indices(src_indices[j], latin_idx2token)
        actual_text = decode_indices(trg_indices[j], devanagari_idx2token)
        predicted_text = decode_indices(output_indices[j], devanagari_idx2token)
        results.append({
            'Input Text': input_text,
            'Actual Text': actual_text,
            'Predicted Text': predicted_text
        })

# Convert results to DataFrame
df = pd.DataFrame(results)

# Save DataFrame to CSV with UTF-8 encoding
csv_file_path = 'seq2seq_predictions_without_attn.csv'
df.to_csv(csv_file_path, index=False, encoding='utf-8')

print('Predictions saved to seq2seq_predictions_without_attn.csv')

# Read and display the CSV file using UTF-8 encoding
df_loaded = pd.read_csv(csv_file_path, encoding='utf-8')
print(df_loaded)

# Generate download link for the CSV file (for use in Jupyter notebook)
def create_download_link(filename):
    import base64
    from IPython.display import HTML
    with open(filename, 'rb') as f:
        b64 = base64.b64encode(f.read()).decode()
    return HTML(f'<a download="{filename}" href="data:file/csv;base64,{b64}">Download {filename}</a>')

# Display the download link (for use in Jupyter notebook)
create_download_link(csv_file_path)


Predictions saved to seq2seq_predictions_without_attn.csv
        Input Text  Actual Text Predicted Text
0        sikhaaega      सिखाएगा        सिखाएगा
1            learn         लर्न         लीरर्न
2         twitters     ट्विटर्स       ट्विटर्स
3      tirunelveli  तिरुनेलवेली   तिरुनेलेवीली
4     independence  इंडिपेंडेंस    इंडीपेंडेंस
...            ...          ...            ...
4090       saflata       सफ़लता         सफलाता
4091        shbana        शबाना          शबाना
4092  khaatootolaa     खातूटॊला       खातूतॊला
4093    shivastava     शिवास्तव       शिवस्तवा
4094  preranapuree  प्रेरणापुरी    प्रेरानपुरी

[4095 rows x 3 columns]


In [15]:
import wandb

In [16]:
wandb.login(key='12c0b23d6865ce943b48c8ea1451c9b2d3aedf60')

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [17]:
# Define the sweep configuration
sweep_config = {
    'method': 'bayes',  # Random search
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'  # Objective: Maximize accuracy
    },
    'parameters': {
        'cell_type': {'values': ['RNN', 'GRU', 'LSTM']},
        'n_enc_layers': {'values': [1, 2, 3]},
        'n_dec_layers': {'values': [1, 2, 3]},
        'hidden_layer_size': {'values': [16, 32, 64, 256]},
        'emb_size': {'values': [16, 32, 64, 256]},
        'bidirectional': {'values': [True,False]},
        'dropout': {'values': [0.2, 0.3]},
        'epochs': {'values': [5, 8, 10]} 
    }
}


# Initialize a new wandb sweep
sweep_id = wandb.sweep(sweep_config, project="DL_Assignment-3")

Create sweep with ID: 3x0wnxao
Sweep URL: https://wandb.ai/lokendrakumar/DL_Assignment-3/sweeps/3x0wnxao


In [18]:
import torch
import torch.nn as nn

def train_model():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Construct run name based on hyperparameters
        run_name = f"{wandb.config.cell_type}_enc-{wandb.config.n_enc_layers}_dec-{wandb.config.n_dec_layers}_hidden-{wandb.config.hidden_layer_size}_emb-{wandb.config.emb_size}_bi-{wandb.config.bidirectional}_drop-{wandb.config.dropout}_epochs-{wandb.config.epochs}"
        wandb.run.name = run_name

        # Initialize the model
        model = build_model(wandb.config.cell_type,
                            wandb.config.n_enc_layers,
                            wandb.config.n_dec_layers,
                            wandb.config.hidden_layer_size,
                            num_encoder_tokens,
                            num_decoder_tokens,
                            wandb.config.dropout,
                            wandb.config.emb_size,
                            device,
                            wandb.config.bidirectional)
        model.to(device)

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()  # Use nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Training and validation loop
        for epoch in range(wandb.config.epochs):
            model.train()
            total_loss = 0
            correct_predictions = 0
            total_predictions = 0

            for input_seqs, target_seqs in tqdm(train_loader, desc="Training"):
                input_seqs, target_seqs = input_seqs.to(device), target_seqs.to(device)
                optimizer.zero_grad()
                output = model(input_seqs, target_seqs)
                loss = criterion(output.view(-1, output.size(-1)), target_seqs.view(-1))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                _, predicted = torch.max(output, 2)
                
                correct_predictions += (predicted == target_seqs).all(dim=1).sum().item()
                total_predictions += target_seqs.size(0)  #numel()

            train_acc = correct_predictions / total_predictions
            train_loss = total_loss / len(train_loader)
            wandb.log({
                "train_loss": train_loss,
                "train_accuracy": train_acc,
            })

            # Validation phase
            model.eval()
            val_total_loss = 0
            val_correct_predictions = 0
            val_total_predictions = 0
            with torch.no_grad():
                for val_input_seqs, val_target_seqs in tqdm(val_loader, desc="Validation"):
                    val_input_seqs, val_target_seqs = val_input_seqs.to(device), val_target_seqs.to(device)
                    val_output = model(val_input_seqs, val_target_seqs)
                    val_loss = criterion(val_output.view(-1, val_output.size(-1)), val_target_seqs.view(-1))
                    val_total_loss += val_loss.item()
                    _, val_predicted = torch.max(val_output, 2)
                    val_correct_predictions += (val_predicted == val_target_seqs).all(dim=1).sum().item()
                    val_total_predictions += val_target_seqs.size(0)  #numel()

            val_acc = val_correct_predictions / val_total_predictions
            val_loss = val_total_loss / len(val_loader)

            # Log metrics to wandb
            wandb.log({
                "val_loss": val_loss,
                "val_accuracy": val_acc
            })
            
wandb.agent(sweep_id, train_model, count=1)  # Run 1 experiment for hyperparameter tuning


wandb: Agent Starting Run: 6eclc2ql with config:
wandb: 	bidirectional: False
wandb: 	cell_type: LSTM
wandb: 	dropout: 0.2
wandb: 	emb_size: 64
wandb: 	epochs: 10
wandb: 	hidden_layer_size: 256
wandb: 	n_dec_layers: 3
wandb: 	n_enc_layers: 1
wandb: Currently logged in as: ma23m008 (lokendrakumar). Use `wandb login --relogin` to force relogin


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/rnn.py:82: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Seq2Seq(
  (encoder): Encoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(100, 64)
    (rnn): LSTM(64, 256, batch_first=True, dropout=0.2)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.2, inplace=False)
    (embedding): Embedding(100, 64)
    (rnn): LSTM(64, 256, num_layers=3, batch_first=True, dropout=0.2)
    (fc): Linear(in_features=256, out_features=100, bias=True)
  )
)


Training:  94%|█████████▎| 749/800 [00:25<00:01, 30.94it/s]wandb: Ctrl + C detected. Stopping sweep.


In [19]:
# Define the sweep configuration
best_sweep = {
    'method': 'bayes',  # Random search
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'  # Objective: Maximize accuracy
    },
    'parameters': {
        'cell_type': {'values': ['LSTM']},
        'n_enc_layers': {'values': [1]},
        'n_dec_layers': {'values': [3]},
        'hidden_layer_size': {'values': [256]},
        'emb_size': {'values': [16]},
        'bidirectional': {'values': [True]},
        'dropout': {'values': [0.2]},
        'epochs': {'values': [10]} 
    }
}


# Initialize a new wandb sweep
best_sweep_id = wandb.sweep(best_sweep, project="DL_Assignment-3")

Training:  94%|█████████▍| 753/800 [00:25<00:01, 28.50it/s]

Create sweep with ID: rgl5iib8
Sweep URL: https://wandb.ai/lokendrakumar/DL_Assignment-3/sweeps/rgl5iib8


In [21]:
import torch
import torch.nn as nn

def train_model():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Construct run name based on hyperparameters
        run_name = f"{wandb.config.cell_type}_enc-{wandb.config.n_enc_layers}_dec-{wandb.config.n_dec_layers}_hidden-{wandb.config.hidden_layer_size}_emb-{wandb.config.emb_size}_bi-{wandb.config.bidirectional}_drop-{wandb.config.dropout}_epochs-{wandb.config.epochs}"
        wandb.run.name = run_name

        # Initialize the model
        model = build_model(wandb.config.cell_type,
                            wandb.config.n_enc_layers,
                            wandb.config.n_dec_layers,
                            wandb.config.hidden_layer_size,
                            num_encoder_tokens,
                            num_decoder_tokens,
                            wandb.config.dropout,
                            wandb.config.emb_size,
                            device,
                            wandb.config.bidirectional)
        model.to(device)

        # Define the loss function and optimizer
        criterion = nn.CrossEntropyLoss()  # Use nn.CrossEntropyLoss()
        optimizer = torch.optim.Adam(model.parameters())

        # Training and validation loop
        for epoch in range(wandb.config.epochs):
            model.train()
            total_loss = 0
            correct_predictions = 0
            total_predictions = 0

            for input_seqs, target_seqs in tqdm(train_loader, desc="Training"):
                input_seqs, target_seqs = input_seqs.to(device), target_seqs.to(device)
                optimizer.zero_grad()
                output = model(input_seqs, target_seqs)
                loss = criterion(output.view(-1, output.size(-1)), target_seqs.view(-1))
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
                
                _, predicted = torch.max(output, 2)
                correct_predictions += (predicted == target_seqs).all(dim=1).sum().item()
                total_predictions += target_seqs.size(0)  #numel()
            train_acc = correct_predictions / total_predictions

            train_acc = correct_predictions / total_predictions
            train_loss = total_loss / len(train_loader)
            wandb.log({
                "train_loss": train_loss,
                "train_accuracy": train_acc,
            })

            # Test phase
            model.eval()
            test_total_loss = 0
            test_correct_predictions = 0
            test_total_predictions = 0
            with torch.no_grad():
                for test_input_seqs, test_target_seqs in tqdm(test_loader, desc="Test"):
                    test_input_seqs, test_target_seqs = test_input_seqs.to(device), test_target_seqs.to(device)
                    test_output = model(test_input_seqs, test_target_seqs)
                    test_loss = criterion(test_output.view(-1, test_output.size(-1)), test_target_seqs.view(-1))
                    test_total_loss += test_loss.item()
 
                    _, test_predicted = torch.max(test_output, 2)
                    test_correct_predictions += (test_predicted == test_target_seqs).all(dim=1).sum().item()
                    test_total_predictions += test_target_seqs.size(0)

            test_acc = test_correct_predictions / test_total_predictions
            test_loss = test_total_loss / len(test_loader)

            # Log metrics to wandb
            wandb.log({
                "test_loss": test_loss,
                "test_accuracy": test_acc
            })
            
wandb.agent(best_sweep_id, train_model, count=1)  